In [1]:
from __future__ import print_function
import torch
import torch.utils.data
from torch import nn, optim
from torch.nn import functional as F
from torchvision import datasets, transforms
from torchvision.utils import save_image
from tensorboardX import SummaryWriter

from tqdm import tqdm

batch_size = 128
epochs = 5
seed = 1
torch.manual_seed(seed)

if torch.cuda.is_available():
    device = "cuda"
else:
    device = "cpu"

In [2]:
from Tars.distributions import Normal, Bernoulli, Categorical
from Tars.losses import KullbackLeibler
from Tars.models import VAE

In [3]:
kwargs = {'num_workers': 1, 'pin_memory': True}
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=True, download=True,
                   transform=transforms.ToTensor()),
    batch_size=batch_size, shuffle=True, **kwargs)
test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=False, transform=transforms.ToTensor()),
    batch_size=batch_size, shuffle=True, **kwargs)

In [4]:
x_dim = 784
y_dim = 10
z_dim = 64


# inference model q(z|x,y)
class Inference(Normal):
    def __init__(self):
        super(Inference, self).__init__(cond_var=["x","y"], var=["z"])

        self.fc1 = nn.Linear(x_dim+y_dim, 512)
        self.fc2 = nn.Linear(512, 512)
        self.fc31 = nn.Linear(512, z_dim)
        self.fc32 = nn.Linear(512, z_dim)

    def forward(self, x, y):
        h = F.relu(self.fc1(torch.cat([x, y], 1)))
        h = F.relu(self.fc2(h))        
        return {"loc": self.fc31(h), "scale": F.softplus(self.fc32(h))}
    
# inference model q(z|x)
class InferenceX(Normal):
    def __init__(self):
        super(InferenceX, self).__init__(cond_var=["x"], var=["z"])

        self.fc1 = nn.Linear(x_dim, 512)
        self.fc2 = nn.Linear(512, 512)
        self.fc31 = nn.Linear(512, z_dim)
        self.fc32 = nn.Linear(512, z_dim)

    def forward(self, x):
        h = F.relu(self.fc1(x))
        h = F.relu(self.fc2(h))        
        return {"loc": self.fc31(h), "scale": F.softplus(self.fc32(h))}
    
# inference model q(z|y)
class InferenceY(Normal):
    def __init__(self):
        super(InferenceY, self).__init__(cond_var=["y"], var=["z"])

        self.fc1 = nn.Linear(y_dim, 512)
        self.fc2 = nn.Linear(512, 512)
        self.fc31 = nn.Linear(512, z_dim)
        self.fc32 = nn.Linear(512, z_dim)

    def forward(self, y):
        h = F.relu(self.fc1(y))
        h = F.relu(self.fc2(h))        
        return {"loc": self.fc31(h), "scale": F.softplus(self.fc32(h))}

    
# generative model p(x|z)    
class GeneratorX(Bernoulli):
    def __init__(self):
        super(GeneratorX, self).__init__(cond_var=["z"], var=["x"])

        self.fc1 = nn.Linear(z_dim, 512)
        self.fc2 = nn.Linear(512, 512)
        self.fc3 = nn.Linear(512, x_dim)

    def forward(self, z):
        h = F.relu(self.fc1(z))
        h = F.relu(self.fc2(h))
        return {"probs": F.sigmoid(self.fc3(h))}
    
# generative model p(y|z)    
class GeneratorY(Categorical):
    def __init__(self):
        super(GeneratorY, self).__init__(cond_var=["z"], var=["y"])

        self.fc1 = nn.Linear(z_dim, 512)
        self.fc2 = nn.Linear(512, 512)
        self.fc3 = nn.Linear(512, y_dim)

    def forward(self, z):
        h = F.relu(self.fc1(z))
        h = F.relu(self.fc2(h))
        return {"probs": F.softmax(self.fc3(h), dim=1)}

    
# prior model p(z)
loc = torch.tensor(0.).to(device)
scale = torch.tensor(1.).to(device)
prior = Normal(loc=loc, scale=scale, var=["z"], dim=z_dim)

In [5]:
p_x = GeneratorX().to(device)
p_y = GeneratorY().to(device)

q = Inference().to(device)
q_x = InferenceX().to(device)
q_y = InferenceY().to(device)

In [6]:
p = p_x * p_y
print(p.prob_text, p.prob_factorized_text)

p(y,x|z) p(y|z)p(x|z)


In [7]:
kl = KullbackLeibler(q, prior)
kl_x = KullbackLeibler(q, q_x)
kl_y = KullbackLeibler(q, q_y)

regularizer = kl + kl_x + kl_y

model = VAE(q, p, other_distributions=[q_x, q_y],
            regularizer=regularizer, optimizer=optim.Adam, optimizer_params={"lr":1e-3})

In [8]:
def train(epoch):
    train_loss = 0
    for batch_idx, (x_data, y_data) in enumerate(tqdm(train_loader)):
        x_data = x_data.to(device)
        y_data = torch.eye(10)[y_data].to(device)        
        lower_bound, loss = model.train({"x": x_data.view(-1, 784), "y": y_data})
        train_loss += loss
 
    train_loss = train_loss * train_loader.batch_size / len(train_loader.dataset)
    print('Epoch: {} Train loss: {:.4f}'.format(epoch, train_loss))
    return train_loss

In [9]:
def test(epoch):
    test_loss = 0
    for i, (x_data, y_data) in enumerate(test_loader):
        x_data = x_data.to(device)
        y_data = torch.eye(10)[y_data].to(device)
        lower_bound, loss = model.test({"x": x_data.view(-1, 784), "y": y_data})
        test_loss += loss

    test_loss = test_loss * test_loader.batch_size / len(test_loader.dataset)
    print('Test loss: {:.4f}'.format(test_loss))
    return test_loss

In [10]:
def plot_reconstrunction_missing(x_data):
    with torch.no_grad():
        z = q_x.sample({"x": x_data.view(-1, 784)}, return_all=False)
        recon_batch = p_x.sample_mean(z).view(-1, 1, 28, 28)
    
        comparison = torch.cat([x_data, recon_batch]).cpu()
        return comparison
    
def plot_image_from_label(x_data, y_data):
    with torch.no_grad():
        x_all = [x_data]
        for i in range(7):
            z = q_y.sample({"y": y_data}, return_all=False)
            recon_batch = p_x.sample_mean(z).view(-1, 1, 28, 28)
            x_all.append(recon_batch)
    
        comparison = torch.cat(x_all).cpu()
        return comparison

def plot_reconstrunction(x_data, y_data):
    with torch.no_grad():
        z = q.sample({"x": x_data.view(-1, 784), "y": y_data}, return_all=False)
        recon_batch = p_x.sample_mean(z).view(-1, 1, 28, 28)
    
        comparison = torch.cat([x_data, recon_batch]).cpu()
        return comparison

In [11]:
writer = SummaryWriter()

plot_number = 1

x_original, y_original = iter(test_loader).next()
x_original = x_original.to(device)
y_original = torch.eye(10)[y_original].to(device)

for epoch in range(1, epochs + 1):
    train_loss = train(epoch)
    test_loss = test(epoch)
    
    recon = plot_reconstrunction(x_original[:8], y_original[:8])
    sample = plot_image_from_label(x_original[:8], y_original[:8])
    recon_missing = plot_reconstrunction_missing(x_original[:8])

    writer.add_scalar('train_loss', train_loss.item(), epoch)
    writer.add_scalar('test_loss', test_loss.item(), epoch)      

    writer.add_image('Image_from_label', sample, epoch)
    writer.add_image('Image_reconstrunction', recon, epoch)    
    writer.add_image('Image_reconstrunction_missing', recon_missing, epoch)
    
writer.close()

100%|██████████| 469/469 [01:25<00:00,  5.47it/s]

Epoch: 1 Train loss: 185.5033



  0%|          | 0/469 [00:00<?, ?it/s]

Test loss: 147.5261


100%|██████████| 469/469 [01:51<00:00,  4.20it/s]


...

Epoch: 4 Train loss: 114.6006


  0%|          | 0/469 [00:00<?, ?it/s]

Test loss: 112.9474


100%|██████████| 469/469 [01:20<00:00,  5.84it/s]


Epoch: 5 Train loss: 110.8761
Test loss: 109.8103
